<a href="https://colab.research.google.com/github/tinLnwza007/Emotion_scan/blob/main/Emotion_YoloModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-8a885c20-5730-de40-6a6a-42c258e6d75a)


In [ ]:
!pip -q install -U ultralytics kaggle opencv-python tqdm "pillow<12.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.4/86.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 132.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.8 MB/s eta 0:00:00


In [ ]:
import PIL
print(PIL.__version__)

11.3.0


# Kaggle API

In [ ]:
from google.colab import files
import os, json, pathlib, stat

print("upload ไฟล์ kaggle.json")
uploaded = files.upload()

assert "kaggle.json" in uploaded, "ไม่พบ kaggle.json"

kaggle_dir = pathlib.Path("/root/.kaggle")
kaggle_dir.mkdir(parents=True, exist_ok=True)

with open(kaggle_dir / "kaggle.json", "wb") as f:
    f.write(uploaded["kaggle.json"])

os.chmod(kaggle_dir / "kaggle.json", stat.S_IRUSR | stat.S_IWUSR)
print("ตั้งค่า Kaggle API สำเร็จ")


upload ไฟล์ kaggle.json


Saving kaggle.json to kaggle.json
ตั้งค่า Kaggle API สำเร็จ


# ดาวน์โหลด Dataset FER2013 (ไฟล์ CSV)

In [ ]:
# แก้ปัญหา ImportError ของ kaggle / kagglesdk บน Colab
!pip -q uninstall -y kaggle kagglesdk
!pip -q install kaggle==1.6.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from pathlib import Path

DATA_ROOT = Path("/content/data")
DATA_ROOT.mkdir(parents=True, exist_ok=True)

# ตรวจสอบ kaggle cli
!kaggle --version

# ดาวน์โหลด FER2013
!kaggle datasets download -d msambare/fer2013 -p {DATA_ROOT} --force

# แสดงไฟล์
!ls -lh {DATA_ROOT}

Kaggle API 1.6.17
Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
 96% 58.0M/60.3M [00:03<00:00, 23.9MB/s]
100% 60.3M/60.3M [00:03<00:00, 16.8MB/s]
total 61M
-rw-r--r-- 1 root root 61M Jul 19  2020 fer2013.zip


In [ ]:
import zipfile
from pathlib import Path

zip_files = list(DATA_ROOT.glob("*.zip"))
assert len(zip_files) > 0, "ไม่พบไฟล์ zip ที่ดาวน์โหลดจาก Kaggle"

zip_path = zip_files[0]
print("Unzipping:", zip_path)

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(DATA_ROOT)

!find {DATA_ROOT} -maxdepth 2 -type f | head -n 50


Unzipping: /content/data/fer2013.zip
/content/data/fer2013.zip


# จัดเรียง data ใหม่

```
/content/fer_yolo_cls/train/<class>/xxx.png
/content/fer_yolo_cls/val/<class>/xxx.png
```





In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm

OUT_ROOT = Path("/content/fer_yolo_cls")
TRAIN_DIR = OUT_ROOT / "train"
VAL_DIR   = OUT_ROOT / "val"

# label mapping มาตรฐาน FER2013 (0-6)
IDX2CLASS = {
    0: "angry",
    1: "disgust",
    2: "fear",
    3: "happy",
    4: "sad",
    5: "surprise",
    6: "neutral"
}

def ensure_dirs():
    for split_dir in [TRAIN_DIR, VAL_DIR]:
        split_dir.mkdir(parents=True, exist_ok=True)
        for _, name in IDX2CLASS.items():
            (split_dir / name).mkdir(parents=True, exist_ok=True)

ensure_dirs()

# 1) หา fer2013.csv
csv_candidates = list(DATA_ROOT.rglob("fer2013.csv"))
if len(csv_candidates) == 0:
    print("ไม่พบ fer2013.csv ในชุดนี้")
else:
    csv_path = csv_candidates[0]
    print("Found CSV:", csv_path)

    df = pd.read_csv(csv_path)
    print(df.head())
    print("Columns:", df.columns.tolist())
    assert "emotion" in df.columns and "pixels" in df.columns, "CSV ไม่ใช่รูปแบบ FER2013 มาตรฐาน"

    #  split ถ้ามี Usage ใช้ Training/PublicTest/PrivateTest
    has_usage = "Usage" in df.columns
    if has_usage:
        # Training -> train, PublicTest/PrivateTest -> val
        split_train = df[df["Usage"].str.contains("Training", case=False, na=False)]
        split_val   = df[~df.index.isin(split_train.index)]
    else:
        # ถ้าไม่มี Usage ให้สุ่มแบ่ง 80/20
        df = df.sample(frac=1, random_state=42).reset_index(drop=True)
        n_train = int(len(df) * 0.8)
        split_train = df.iloc[:n_train]
        split_val   = df.iloc[n_train:]

    def save_images(split_df, split_dir: Path, prefix: str):
        for i, row in tqdm(split_df.iterrows(), total=len(split_df)):
            emo = int(row["emotion"])
            cls = IDX2CLASS.get(emo, f"class_{emo}")

            pixels = np.fromstring(row["pixels"], sep=" ", dtype=np.uint8)
            if pixels.size != 48*48:
                # ข้ามกรณีข้อมูลเสียหาย
                continue
            img = pixels.reshape(48, 48)

            # แปลงเป็น RGB 3 channels เพื่อความเข้ากันได้กับโมเดลทั่วไป
            pil = Image.fromarray(img, mode="L").convert("RGB")

            out_path = split_dir / cls / f"{prefix}_{i:06d}.png"
            pil.save(out_path, optimize=True)

    # ลบของเก่าถ้ามี (กันซ้ำ)
    for d in [TRAIN_DIR, VAL_DIR]:
        for p in d.rglob("*.png"):
            p.unlink()

    save_images(split_train, TRAIN_DIR, "train")
    save_images(split_val,   VAL_DIR,   "val")

    print("เสร็จสิ้นการจัดโครงสร้าง dataset:", OUT_ROOT)

# แสดงตัวอย่างโครงสร้าง
!find {OUT_ROOT} -maxdepth 2 -type d | sort | head -n 50
!find {OUT_ROOT} -type f | head -n 10


ไม่พบ fer2013.csv ในชุดนี้
/content/fer_yolo_cls
/content/fer_yolo_cls/train
/content/fer_yolo_cls/train/angry
/content/fer_yolo_cls/train/disgust
/content/fer_yolo_cls/train/fear
/content/fer_yolo_cls/train/happy
/content/fer_yolo_cls/train/neutral
/content/fer_yolo_cls/train/sad
/content/fer_yolo_cls/train/surprise
/content/fer_yolo_cls/val
/content/fer_yolo_cls/val/angry
/content/fer_yolo_cls/val/disgust
/content/fer_yolo_cls/val/fear
/content/fer_yolo_cls/val/happy
/content/fer_yolo_cls/val/neutral
/content/fer_yolo_cls/val/sad
/content/fer_yolo_cls/val/surprise


# สร้างไฟล์ classes.json เอาไปใช้ในเว็บ

In [ ]:
import json
from pathlib import Path

classes = [IDX2CLASS[i] for i in range(7)]
classes_path = Path("/content/classes.json")
with open(classes_path, "w", encoding="utf-8") as f:
    json.dump(classes, f, ensure_ascii=False)

print("Saved:", classes_path)
print("Classes:", classes)


Saved: /content/classes.json
Classes: ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']




```
# This is formatted as code
```

# Train YOLO Classification
YOLO11n-cls หรือ YOLOv8n-cls

> https://docs.ultralytics.com/modes/train/#apple-silicon-mps-training



In [ ]:
from pathlib import Path

root = Path("/content/fer_yolo_cls")
assert root.exists()
assert (root/"train").exists()
assert (root/"val").exists()

n_train = sum(1 for _ in (root/"train").rglob("*.*"))
n_val   = sum(1 for _ in (root/"val").rglob("*.*"))
print("Train:", n_train)
print("Val:", n_val)
print("Dataset ready")

Train: 0
Val: 0
Dataset ready


In [ ]:
from ultralytics import YOLO
from pathlib import Path

DATA_DIR = Path("/content/fer_yolo_cls")

# เลือก yolo11n-cls.pt
backbones = ["yolo11n-cls.pt", "yolov8n-cls.pt"]

model = None
chosen = None
for bb in backbones:
    try:
        model = YOLO(bb)
        chosen = bb
        break
    except Exception as e:
        print(f"โหลด {bb} ไม่สำเร็จ -> {e}")

assert model is not None, "ไม่สามารถโหลดโมเดลได้"

print("ใช้ backbone:", chosen)

# เทรน
results = model.train(
    data=str(DATA_DIR),   # โฟลเดอร์ที่มี train/ และ val/
    epochs=10,            # แนะนำ 10-20 สำหรับเดโม
    imgsz=64,             # FER2013 ขนาดเล็ก
    batch=64,
    device=0              # ใช้ GPU ถ้ามี
)

print("Training finished.")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
ใช้ backbone: yolo11n-cls.pt
Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/fer_yolo_cls, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=64, int8=False

RuntimeError: Dataset '/content/fer_yolo_cls' error ❌ /content/fer_yolo_cls 'train:' no training images found

# ถ้าหา dataset ไม่เจอ รันด้านล่างตรวจดู

In [ ]:
!ls -lah /content
!ls -lah /content/fer_yolo_cls
!find /content/fer_yolo_cls -maxdepth 2 -type d | sort | head -n 50
!find /content/fer_yolo_cls -type f | head -n 20


total 5.6M
drwxr-xr-x 1 root root 4.0K Jan  8 08:00 .
drwxr-xr-x 1 root root 4.0K Jan  8 07:22 ..
drwxr-xr-x 4 root root 4.0K Dec  9 14:41 .config
drwxr-xr-x 4 root root 4.0K Jan  8 07:38 data
drwxr-xr-x 4 root root 4.0K Jan  8 07:43 fer_yolo_cls
-rw-r--r-- 1 root root   62 Jan  8 07:31 kaggle.json
drwxr-xr-x 3 root root 4.0K Jan  8 08:00 runs
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data
-rw-r--r-- 1 root root 5.6M Jan  8 08:00 yolo11n-cls.pt
total 16K
drwxr-xr-x 4 root root 4.0K Jan  8 07:43 .
drwxr-xr-x 1 root root 4.0K Jan  8 08:00 ..
drwxr-xr-x 9 root root 4.0K Jan  8 07:43 train
drwxr-xr-x 9 root root 4.0K Jan  8 07:43 val
/content/fer_yolo_cls
/content/fer_yolo_cls/train
/content/fer_yolo_cls/train/angry
/content/fer_yolo_cls/train/disgust
/content/fer_yolo_cls/train/fear
/content/fer_yolo_cls/train/happy
/content/fer_yolo_cls/train/neutral
/content/fer_yolo_cls/train/sad
/content/fer_yolo_cls/train/surprise
/content/fer_yolo_cls/val
/content/fer_yolo_cls/val/angry
/conte

In [ ]:
!find /content/fer_yolo_cls/train -type f | wc -l
!find /content/fer_yolo_cls/val -type f | wc -l


0
0


In [ ]:
!find /content/data -maxdepth 2 -type f | head -n 50
!find /content/data -maxdepth 2 -type d | sort | head -n 50

/content/data/fer2013.zip
/content/data
/content/data/test
/content/data/test/angry
/content/data/test/disgust
/content/data/test/fear
/content/data/test/happy
/content/data/test/neutral
/content/data/test/sad
/content/data/test/surprise
/content/data/train
/content/data/train/angry
/content/data/train/disgust
/content/data/train/fear
/content/data/train/happy
/content/data/train/neutral
/content/data/train/sad
/content/data/train/surprise


In [ ]:
!rm -rf /content/fer_yolo_cls
!mkdir -p /content/fer_yolo_cls

!cp -r /content/data/train /content/fer_yolo_cls/train
!cp -r /content/data/test  /content/fer_yolo_cls/val

In [ ]:
!find /content/fer_yolo_cls/train -type f | head -n 5
!find /content/fer_yolo_cls/val -type f | head -n 5
!find /content/fer_yolo_cls/train -type f | wc -l
!find /content/fer_yolo_cls/val -type f | wc -l

/content/fer_yolo_cls/train/happy/Training_25540366.jpg
/content/fer_yolo_cls/train/happy/Training_1278231.jpg
/content/fer_yolo_cls/train/happy/Training_83846983.jpg
/content/fer_yolo_cls/train/happy/Training_2903091.jpg
/content/fer_yolo_cls/train/happy/Training_32942972.jpg
/content/fer_yolo_cls/val/happy/PublicTest_82875599.jpg
/content/fer_yolo_cls/val/happy/PublicTest_69166455.jpg
/content/fer_yolo_cls/val/happy/PrivateTest_24030799.jpg
/content/fer_yolo_cls/val/happy/PublicTest_36291152.jpg
/content/fer_yolo_cls/val/happy/PublicTest_10245799.jpg
28709
7178


In [ ]:
from pathlib import Path

root = Path("/content/fer_yolo_cls")
assert root.exists()
assert (root/"train").exists()
assert (root/"val").exists()

n_train = sum(1 for _ in (root/"train").rglob("*.*"))
n_val   = sum(1 for _ in (root/"val").rglob("*.*"))
print("Train:", n_train)
print("Val:", n_val)
print("Dataset ready")


Train: 28709
Val: 7178
Dataset ready


# ลองสร้าง Train Model

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n-cls.pt")  # ใช้ตัวที่โหลดสำเร็จแล้ว
model.train(
    data="/content/fer_yolo_cls",
    epochs=10,      # เดโมในคลาส 10-20 พอ
    imgsz=64,
    batch=64,
    device=0
)

Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/fer_yolo_cls, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=64, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, p

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d516b43cf20>
curves: []
curves_results: []
fitness: 0.8039844036102295
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.6200891733169556, 'metrics/accuracy_top5': 0.9878796339035034, 'fitness': 0.8039844036102295}
save_dir: PosixPath('/content/runs/classify/train2')
speed: {'preprocess': 0.009083442602769585, 'inference': 0.14703458595714441, 'loss': 0.0004991950403501922, 'postprocess': 0.000405248119164529}
task: 'classify'
top1: 0.6200891733169556
top5: 0.9878796339035034

# Export เป็น ONNX + สร้างไฟล์ classes.json เอาไปใช้ใน Next.js

In [ ]:
from pathlib import Path

runs_dir = Path("/content/runs/classify")
train_dirs = sorted([p for p in runs_dir.glob("train*") if p.is_dir()],
                    key=lambda p: p.stat().st_mtime, reverse=True)
assert len(train_dirs) > 0, "ไม่พบโฟลเดอร์ train ใน runs/classify"

last_train = train_dirs[0]
best_pt = last_train / "weights" / "best.pt"
print("Best:", best_pt)
assert best_pt.exists(), "ไม่พบ best.pt"

Best: /content/runs/classify/train2/weights/best.pt


In [ ]:
from ultralytics import YOLO
from pathlib import Path

m = YOLO(str(best_pt))
m.export(format="onnx", opset=12)

# หาไฟล์ onnx ที่สร้าง
onnx_files = list(best_pt.parent.glob("*.onnx"))
if not onnx_files:
    onnx_files = list(best_pt.parent.parent.glob("*.onnx"))

print("ONNX:", onnx_files)
assert len(onnx_files) > 0, "ไม่พบไฟล์ ONNX หลัง export"

final_onnx = Path("/content/emotion_yolo11n_cls.onnx")
onnx_files[0].rename(final_onnx)
print("Saved:", final_onnx)


Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11n-cls summary (fused): 47 layers, 1,534,991 parameters, 0 gradients, 3.2 GFLOPs

PyTorch: starting from '/content/runs/classify/train2/weights/best.pt' with input shape (1, 3, 64, 64) BCHW and output shape(s) (1, 7) (3.1 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<2.0.0', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 151ms
Prepared 6 packages in 6.78s
Installed 6 packages in 266ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.20.0
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.82

requirements: AutoUpdate success ✅ 7.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting e

In [ ]:
import json
from pathlib import Path

train_root = Path("/content/fer_yolo_cls/train")
classes = sorted([p.name for p in train_root.iterdir() if p.is_dir()])
print("Classes:", classes)

classes_path = Path("/content/classes.json")
with open(classes_path, "w", encoding="utf-8") as f:
    json.dump(classes, f, ensure_ascii=False)

print("Saved:", classes_path)

Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
Saved: /content/classes.json


# แพ็กไฟล์ไปใช้ต่อบนเว็บ .zip

In [ ]:
import zipfile
from pathlib import Path

final_onnx = Path("/content/emotion_yolo11n_cls.onnx")
classes_path = Path("/content/classes.json")

zip_out = Path("/content/web_models.zip")
with zipfile.ZipFile(zip_out, "w", zipfile.ZIP_DEFLATED) as z:
    z.write(final_onnx, arcname="emotion_yolo11n_cls.onnx")
    z.write(classes_path, arcname="classes.json")

print("Created:", zip_out)

Created: /content/web_models.zip


# Download Zip File

In [ ]:
from google.colab import files
files.download("/content/web_models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>